In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_excel("Glob_Superstore.xlsx")

In [ ]:
df.to_csv("Glob_Superstore.csv", index = None, header = True)                    #converting excel file to csv
store_df = pd.DataFrame(pd.read_csv("Glob_Superstore.csv"))

In [ ]:
store_df.columns

In [ ]:
store_df.describe

## Data Cleaning

In [ ]:
#Checking for null values
store_df.info()

In [ ]:
#We shall drop the "Postal Code" column since it is not needed for this analysis (and also has null values)
store_df = store_df.drop(columns = ['Postal Code'])

In [ ]:
#Find any duplicates
store_df.duplicated().sum()

In [ ]:
#Converting the dates in the Order Date and Ship Date columns to proper datetime values
store_df["Order Date"] = pd.to_datetime(store_df["Order Date"], errors = 'coerce')
store_df["Ship Date"] = pd.to_datetime(store_df["Ship Date"], errors = 'coerce')

In [ ]:
store_df

## Exploratory Data Analysis

In [ ]:
#Finding the distribution of Shipping Segments
segment_counts = store_df["Segment"].value_counts()
colors = sns.color_palette('bright')[0:5]
plt.pie(segment_counts.values, labels = segment_counts.index, colors = colors, autopct='%.0f%%')
plt.title("Proportion of Segments Shipped")
plt.show()

This pie chart shows that Consumer Goods consitute the highest amount shipped while Home Office Supplies were the least.

In [ ]:
#Finding the distribution of Shipping Modes
shpmode_counts = store_df["Ship Mode"].value_counts()
colors = sns.color_palette('bright')[0:6]
plt.pie(shpmode_counts.values, labels = shpmode_counts.index, colors = colors, autopct='%.0f%%')
plt.title("Proportion of Ship Modes")
plt.show()

This pie chart shows that Standard Class consitute the highest mode shipped while Same Day modes were the least.

In [ ]:
#Counting the number of observations per Categories 
ax = sns.countplot(store_df['Category'])
plt.show()

## Plotting Sales vs Profit Graph for each Category

In [ ]:
data = store_df.copy()                                                     #create a copy of the dataframe

In [ ]:
group_category = data.groupby("Category")
group_category.head()

In [ ]:
cat_profit = data.groupby("Category")['Profit']                             
cat_sales = data.groupby("Category")['Sales']

In [ ]:
pr_sales = data.groupby("Category")['Profit','Sales']
pr_sales.sum()                                                     #create a new df showing the total profit/sales per category

In [ ]:
pr_sales.agg(['sum']).plot.bar()
plt.title('Total Profit and Sales per Category')
plt.ylabel('Sales/Profit')
plt.show()

Technology products were the most sold while Office Supplies were the least sold. In addition, Technology products were also the most profitable, while Furniture products were the least profitable.

## Which Category Shows the Highest Profit?

In [ ]:
high_cat_pr = data.groupby(by=['Category']).sum().sort_values(by=['Profit']).reset_index()
high_cat_pr

In [ ]:
plt.bar("Category",'Profit',data=high_cat_pr)
plt.show()

## Visualizing Total Profits

In [ ]:
data['Year'] = data['Order Date'].dt.year                #extracting the order year 
data['Month'] = data['Order Date'].dt.month              #extracting the order month 
data['Day'] = data['Order Date'].dt.day                  #extracting the order day 
data["Month_year"] = data['Order Date'].apply(lambda x: x.strftime('%Y-%m'))

In [ ]:
#Grouping the profits into month and year
data_m_yr = data.groupby('Month_year').sum()['Profit'].reset_index()

In [ ]:
data_m_yr

In [ ]:
#Plot the monthly trend of Profits
import matplotlib.dates as mdates 
fig, ax = plt.subplots()
ax.plot(data_m_yr["Month_year"], data_m_yr["Profit"])
locator = mdates.DayLocator(interval=10)
ax.xaxis.set_major_locator(locator)                         
ax.set_xlabel("Date")
ax.set_ylabel("Profit")
plt.show()

We can see that the Total Profits have increased over the given time period, even though it fluctuates from time to time. 

## Which Order Had the Highest Sales?

In [ ]:
#Grouping the sales based on Order Priority
data_order_sales = data.groupby('Order Priority').mean()['Sales'].reset_index()

In [ ]:
data_order_sales

In [ ]:
#Ropunding the values in the Sales column
data_order_sales['Sales'] = data_order_sales['Sales'].round(2)

# find the index for the max Sales
max_sales = data_order_sales['Sales'].idxmax()

# get the Order Priority value for the max Profit index
order_max = data_order_sales.loc[max_sales, 'Order Priority']

# print the statement
print(f"{order_max} order priority had the highest sales at US$ {data_order_sales['Sales'].max()}.")